In [2]:
!pip install numpy pandas

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import random

In [4]:
data_path = '/kaggle/input/roman-urdu-poetry/Roman-Urdu-Poetry.csv'
df = pd.read_csv(data_path)

poetry_texts = df['Poetry'].dropna().tolist()  


all_text = "\n".join(poetry_texts)

In [5]:
chars = sorted(list(set(all_text)))
vocab_size = len(chars)

char2idx = {ch: idx for idx, ch in enumerate(chars)}
idx2char = {idx: ch for idx, ch in enumerate(chars)}

def text_to_int(text):
    return [char2idx[ch] for ch in text]

def int_to_text(indices):
    return ''.join([idx2char[idx] for idx in indices])

all_data = text_to_int(all_text)


In [6]:
class PoetryDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        x = self.data[idx : idx + self.seq_length]
        y = self.data[idx + 1 : idx + self.seq_length + 1]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

SEQ_LENGTH = 256  # you can adjust this
BATCH_SIZE = 256

full_dataset = PoetryDataset(all_data, SEQ_LENGTH)

In [7]:
dataset_size = len(full_dataset)
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)


In [9]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(CharRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)  
        output, hidden = self.lstm(x, hidden)  
        output = output.contiguous().view(-1, output.shape[2])  
        logits = self.fc(output)  
        return logits, hidden

EMBED_SIZE = 128
HIDDEN_SIZE = 256
NUM_LAYERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.003

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CharRNN(vocab_size, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [10]:
model.train()  
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        logits, _ = model(inputs)
        loss = criterion(logits, targets.view(-1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}")

Epoch [1/10], Loss: 0.9422
Epoch [2/10], Loss: 0.4361
Epoch [3/10], Loss: 0.3530
Epoch [4/10], Loss: 0.3243
Epoch [5/10], Loss: 0.3085
Epoch [6/10], Loss: 0.2983
Epoch [7/10], Loss: 0.2904
Epoch [8/10], Loss: 0.2849
Epoch [9/10], Loss: 0.2800
Epoch [10/10], Loss: 0.2761


In [11]:
model_path = 'char_rnn_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_size': EMBED_SIZE,
    'hidden_size': HIDDEN_SIZE,
    'num_layers': NUM_LAYERS,
    'char2idx': char2idx,
    'idx2char': idx2char,
}, model_path)
print(f"Model saved to {model_path}")


Model saved to char_rnn_model.pth


In [12]:
def load_model(model_path, device):
    checkpoint = torch.load(model_path, map_location=device)
    loaded_model = CharRNN(checkpoint['vocab_size'],
                           checkpoint['embed_size'],
                           checkpoint['hidden_size'],
                           checkpoint['num_layers']).to(device)
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    loaded_model.eval() 
    char2idx = checkpoint['char2idx']
    idx2char = checkpoint['idx2char']
    return loaded_model, char2idx, idx2char

loaded_model, loaded_char2idx, loaded_idx2char = load_model(model_path, device)
print("Model loaded for testing or generation.")

Model loaded for testing or generation.


<ipython-input-12-92d340061674>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


In [13]:
def generate_text(model, start_text, char2idx, idx2char, generation_length=200, temperature=0.8):
    model.eval()
    input_indices = [char2idx.get(ch, 0) for ch in start_text]
    input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0).to(device)

    hidden = None
    generated_text = start_text

    for _ in range(generation_length):
        logits, hidden = model(input_tensor, hidden)
        logits = logits[-1] / temperature
        probabilities = torch.softmax(logits, dim=0).detach().cpu().numpy()
        next_char_idx = np.random.choice(len(probabilities), p=probabilities)

        next_char = idx2char[next_char_idx]
        generated_text += next_char

        input_tensor = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

    return generated_text

prompt = "pyar" 
generated_poetry = generate_text(loaded_model, prompt, loaded_char2idx, loaded_idx2char)
print("Generated Poetry:\n", generated_poetry)

Generated Poetry:
 pyar kahe baġhair mujhe 
betāb parī-dār-e-rahguzar tarāhat par 
shaam bhī Daal kar jitnā jahāñ se dar hī chukā 
thā jurm meñhī bhalā jaane hai jaan denā samajh baiThe the ham 
āñsuoñ kā hisāb dosto kahīñ 
